In [1]:
import numpy as np 
import os
import tensorflow as tf
from tensorflow.python.framework import graph_util

#construct data
x_data = np.linspace(-1,1,300, dtype=np.float32)
noise = np.random.normal(0, 0.05, x_data.shape).astype(np.float32)
y_data = 2 * x_data - 1 + noise

#slice train data
train_x = x_data[0:200]
train_y = y_data[0:200]

#slice test data
test_x = x_data[200:]
test_y = y_data[200:]

# Parameters
learning_rate = 0.01
training_epochs = 100
display_step = 50

with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, name="x")
    Y = tf.placeholder(tf.float32, name="y")

with tf.name_scope('my-model'):
    W = tf.Variable(np.random.randn(), name="weight", dtype=tf.float32)
    b = tf.Variable(np.random.randn(), name="bias", dtype=tf.float32)
    pre_y = tf.add(tf.multiply(X, W), b, name="my")

with tf.name_scope('loss'):
    loss = tf.math.reduce_sum(tf.math.pow(pre_y - Y, 2))

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()
print("start")
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        for (x, y) in zip(train_x, train_y):
            sess.run(optimizer, feed_dict={X: x, Y: y})
        #sess.run(optimizer, feed_dict={X: train_x, Y:train_y})
    
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            c = sess.run(loss, feed_dict={X: train_x, Y:train_y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b))

    print("Optimization Finished!")
    training_cost = sess.run(loss, feed_dict={X: train_x, Y: train_y})
    print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')
    
#     print("Testing... (Mean square loss Comparison)")
#     testing_cost = sess.run(loss,feed_dict={X: test_x, Y: test_y})  # same function as cost above
#     print("Testing cost=", testing_cost)
#     print("Absolute mean square loss difference:", abs(
#         training_cost - testing_cost))
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['my-model/my'])
    with tf.gfile.FastGFile(os.getcwd()+'/linear_regression/model.pb', mode='wb') as f:
        f.write(constant_graph.SerializeToString())
    # 生成一个具有写权限的日志文件操作对象，将当前命名空间的计算图写进日志中
    writer=tf.summary.FileWriter('linear_regression/logs', sess.graph)
    writer.close()

start
Epoch: 0050 cost= 0.489154786 W= 1.98484 b= -1.0015569
Epoch: 0100 cost= 0.489154786 W= 1.98484 b= -1.0015569
Optimization Finished!
Training cost= 0.4891548 W= 1.98484 b= -1.0015569 

INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
Instructions for updating:
Use tf.gfile.GFile.


In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
import os

sess = tf.Session()
with gfile.FastGFile(os.getcwd()+'/linear_regression/model.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='') # 导入计算图

# 需要有一个初始化的过程    
sess.run(tf.global_variables_initializer())

# 需要先复原变量
print(sess.run('model/weight:0'))
print(sess.run('model/bias:0'))
# 1

# 输入
input_x = sess.graph.get_tensor_by_name('input/x:0')

op = sess.graph.get_tensor_by_name('model/my:0')

ret = sess.run(op,  feed_dict={input_x: 5})
print(ret)
# 输出 26
writer=tf.summary.FileWriter('linear_regression/logs', sess.graph)
writer.close()

Instructions for updating:
Use tf.gfile.GFile.


In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
import os

sess = tf.Session()
with gfile.FastGFile('/home/ubuntu/code/tensorflow/models/research/object_detection/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='') # 导入计算图

# # 需要有一个初始化的过程    
# sess.run(tf.global_variables_initializer())

# # 需要先复原变量
# print(sess.run('model/weight:0'))
# print(sess.run('model/bias:0'))
# # 1

# # 输入
# input_x = sess.graph.get_tensor_by_name('input/x:0')

# op = sess.graph.get_tensor_by_name('model/my:0')

# ret = sess.run(op,  feed_dict={input_x: 5})
# print(ret)
# 输出 26
writer=tf.summary.FileWriter('linear_regression/logs', sess.graph)
writer.close()

Instructions for updating:
Use tf.gfile.GFile.
